In [ ]:
import pyaudio
import queue
import threading
import numpy as np
import math
import wave
import matplotlib.pyplot as plt
from scipy.io import wavfile



class MicArray:

    def __init__(self,device_index = None,rate = 16000,channels = 4,chunk_size = 1024):
        self.p = pyaudio.PyAudio()
        self.q = queue.Queue()
        self.thread_event = threading.Event()

        self.rate = rate
        self.channels = channels
        self.chunk_size = chunk_size
        
        if device_index == None:
            for i in range(self.p.get_device_count()):
                dev = self.p.get_device_info_by_index(i)
                name = dev['name'].encode('utf-8')
                print(i, name, dev['maxInputChannels'], dev['maxOutputChannels'])
                if dev['maxInputChannels'] == self.channels:
                    print('Use {}'.format(name))
                    device_index = i
                    break

                
        self.stream = self.p.open(
            input_device_index = device_index,
            start = False,
            format=pyaudio.paInt16,
            channels=self.channels,
            rate=self.rate,
            input=True,
            frames_per_buffer=self.chunk_size,
            stream_callback=self._callback
        )
               
    def _callback(self,input_data,frame_count,time_info,status_flag):
        self.q.put(input_data)
        return (None,pyaudio.paContinue)
    
    def start(self):
        self.q.queue.clear()
        self.stream.start_stream()
    
    def stop(self):
        self.thread_event.set()
        self.stream.stop_stream()
        self.q.put('')

    def read_mic_data(self):
        self.thread_event.clear()
        while not self.thread_event.is_set():
            frames = self.q.get()
            if not frames:
                break
            
            frames = np.frombuffer(frames,dtype = 'int16')
            yield frames

    def __enter__(self):
        self.start()
        return self

    def __exit__(self,exception_type,exception_value,traceback):
        if exception_value:
            return False
        self.stop()
        

def gccphat(sig,refsig,fs,max_tau,interp=1):
    
    n = sig.shape[0] + refsig.shape[0]

    SIG = np.fft.rfft(sig, n=n)
    REFSIG = np.fft.rfft(refsig, n=n)
    R = SIG * np.conj(REFSIG)

    cc = np.fft.irfft(R / np.abs(R), n=(interp * n))
    
    max_shift = np.minimum(int(interp * fs * max_tau), int(interp * n/2))
    
    cc = np.concatenate((cc[-max_shift:], cc[:max_shift+1]))
    
    shift = np.argmax(np.abs(cc)) - max_shift
    
    tau = shift / float(interp * fs)
    
    return tau,cc
    
    

In [ ]:
def find_direction_gccphat(data_frame,fs,tau_mic_dist):
    tau,_ = gccphat(data_frame[0::4],data_frame[3::4],fs=fs,max_tau=tau_mic_dist,interp = 16)
    #print(tau)
    #print(tau_mic_dist)
    #print(tau/tau_mic_dist)
    #print(math.asin(tau/tau_mic_dist))
    theta = math.asin(tau/tau_mic_dist) * 180 / math.pi 
    
    return theta+90

def find_direction_gccphat2(data_frame1,data_frame2,fs,tau_mic_dist):
    tau,_ = gccphat(data_frame1,data_frame2,fs=fs,max_tau=tau_mic_dist)
    theta = math.asin(tau/tau_mic_dist) * 180 / math.pi      
    
    return theta+90

def real_time_separate_mic():
    import signal
    import time
    
    distance = 0.0568
    max_time = distance/343.2
    Sampling_Rate = 16000
    
    is_quit = threading.Event()
    
    def signal_handler(sig, num):
        is_quit.set()
        print('Exited')
    
    signal.signal(signal.SIGINT, signal_handler)
    
    with MicArray(device_index=1,rate=Sampling_Rate,channels=1) as mic1, MicArray(device_index=2,rate=Sampling_Rate,channels=1) as mic2:
        for chunk1,chunk2 in zip(mic1.read_mic_data(),mic2.read_mic_data()):
            direction = find_direction_gccphat2(chunk1,chunk2,fs=Sampling_Rate,tau_mic_dist=max_time)
            print(int(direction))
            
            if is_quit.is_set():
                break

def mic_array_test():
    import signal
    import time
    
    distance = 0.0568
    max_time = distance/343.2
    
    is_quit = threading.Event()
    
    def signal_handler(sig, num):
        is_quit.set()
        print('Exited')
    
    signal.signal(signal.SIGINT, signal_handler)
    
    with MicArray(channels=4) as mic:
        for chunk in mic.read_mic_data():
            chunk = filterData(16000, chunk,61,cutoffFreq = 4000,
                               windowName = "hanning",
                               showFilterResponse = False,
                               saveToFile = False)
            direction = find_direction_gccphat(chunk,fs=mic.rate,tau_mic_dist=max_time)
            print(int(direction))
            
            if is_quit.is_set():
                break


def from_wave_files():
    
    RATE = 16000
    distance = 0.0568
    max_time = distance/343.2
    
    wav = wave.open("data.wav")
    wav.setpos(0)
    sdata = wav.readframes(wav.getnframes())
    
    
    data = np.frombuffer(sdata, dtype=np.int16)
    direction = find_direction_gccphat(data,fs=RATE,tau_mic_dist=max_time)
    print(int(direction))
    

def from_two_files():
    RATE = 16000
    distance = 0.0568
    max_time = distance/343.2
    
    _, data1 = wavfile.read('./ch1.wav') 
    _, data2 = wavfile.read('./ch4.wav')
    
    direction = find_direction_gccphat2(data1,data2,fs=RATE,tau_mic_dist=max_time)
    print(int(direction))
    
def main():
    mic_array_test()
    #from_wave_files()
    #from_two_files()
    #real_time_separate_mic()
    
if __name__ == "__main__":
    main()
    

### Audio spectrum analyser

In [ ]:
def audio_spectrum_analyser(fs = 16000, CHUNK=1024, signal_amp = 32767):
    import signal
    import time
    from tkinter import TclError
        
    is_quit = threading.Event()
    
    def signal_handler(sig, num):
        is_quit.set()
        print('Exited')
    
    signal.signal(signal.SIGINT, signal_handler)
    
    print('stream started')

    # for measuring frame rate
    frame_count = 0
    start_time = time.time()
    
    %matplotlib tk
    fig, (ax1, ax2) = plt.subplots(2)
    x = np.arange(0, CHUNK, 1)
    x_fft = np.linspace(0, int(fs/2), int((CHUNK/2)+1))
    ax1.set_ylim(-signal_amp, signal_amp)
    #ax2.sey_ylim
    line_x, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=1)
    line_fft, = ax2.semilogx(x_fft, np.random.rand(int(CHUNK/2)+1), '-', lw=1)
    
    with MicArray(channels=4) as mic:
        for chunk in mic.read_mic_data():
            
            signal = chunk[0::4]
            y_fft = (np.abs(np.fft.rfft(signal)))*(2/(32767*513))
    
            line_x.set_ydata(signal)      
            line_fft.set_ydata(y_fft)
            
            try:
                fig.canvas.draw()
                fig.canvas.flush_events()
                frame_count += 1
            except TclError:
                frame_rate = frame_count / (time.time() - start_time)
                print('stream stopped')
                print('average frame rate = {:.0f} FPS'.format(frame_rate))
                break
    
            if is_quit.is_set():
                break

In [ ]:
audio_spectrum_analyser()